#### In this step, we want to transform the time series data into Time-series data

In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
rides = pd.read_parquet('../data/transformed/validated_rides_2022_01.parquet')
rides.head(5)

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [3]:
## Then we'll add another column to the dataframe, whic represents the rounded hour for each hour

In [4]:
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
rides

/tmp/ipykernel_32215/2261653417.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')


,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-01 00:35:40,142,2022-01-01 00:00:00
1,2022-01-01 00:33:43,236,2022-01-01 00:00:00
2,2022-01-01 00:53:21,166,2022-01-01 00:00:00
3,2022-01-01 00:25:21,114,2022-01-01 00:00:00
4,2022-01-01 00:36:48,68,2022-01-01 00:00:00
...,...,...,...
2463926,2022-01-31 23:36:53,90,2022-01-31 23:00:00
2463927,2022-01-31 23:44:22,107,2022-01-31 23:00:00
2463928,2022-01-31 23:39:00,113,2022-01-31 23:00:00
2463929,2022-01-31 23:36:42,148,2022-01-31 23:00:00


In [5]:
### Now we want to group, per pickup hour, and per location id. So that we know the number of rides per location id per pickup hour

In [6]:
agg_rides = rides.groupby(['pickup_hour','pickup_location_id']).size().reset_index()

In [8]:
agg_rides.rename(columns= {
    0: 'rides'
},inplace = True)
agg_rides.head(2)

,pickup_hour,pickup_location_id,rides
0,2022-01-01,4,11
1,2022-01-01,7,6


In [13]:
## What we want to make sure though, is that the entire time-series is complete, so that each day has data, for example, we might be having data 
## in the first day, but during some hour on that day, there is no data, we want to make sure that it is filled with 0 for those location IDs.

In [ ]:
def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(agg_rides['pickup_hour'].min, agg_rides['pickup_hour'].max(), freq='H')
    output = pd.DataFrame()

    for location_id in tqdm(location_ids):
        ## keep only the rides for this location id
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id,['pickup_hour','rides']]

        ## Quickest way to add missing values with 0 in a series
        ## Taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour',inplace = True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value = 0)
        
        ## Now add back the location_id columns
        agg_rides_i['pickup_location_id'] = location_id
        output = pd.concat([output, agg_rides_i])

    output = output.reset_index().rename(columns = {'index': 'pickup_hour'})

